In [ ]:
#----------------HH1 & HH2---------------------------------#
#----------------Do cross correlation----------------------#
#----------envelope cross-crrelation method----------------#
import obspy
from obspy.clients.fdsn import Client
client = Client("USGS")
from obspy import UTCDateTime
import numpy as np
from obspy.geodetics import locations2degrees
from obspy.geodetics import gps2dist_azimuth
from obspy.geodetics import degrees2kilometers
from obspy import read

import scipy
import scipy.signal as scisignal
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

from PIL import Image
from IPython.display import Image
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger

import os
# import geopy.distance
import glob
import obspy.signal.filter
import pandas as pd
import glob
import csv
import pickle
import datetime
import copy

In [ ]:
#--------------load OBS data--------------------#
obs = pd.read_csv('Seismometer_station',sep ='   ',header=None)

In [ ]:
#---------A function to calculate segment and cumulate length--------#
def seg_cum_function(data):
    avg = np.mean(data)
    data_box = np.zeros(3*60)
    data_box[data>avg] = 1
    cum_len = np.where(data>avg)
    seg = 0
    for iseg in range(3*60-1):
        if data_box[iseg]==0 and data_box[iseg+1]==1:
            seg += 1   
    return seg,cum_len

In [ ]:
#-------A function to remove zero valus---------------------##
def zeromean(data):
    for iw in range(len(data)):
        if data[iw] == 0.0:
            data[iw] = np.nan
    return np.nanmean(data)
        

In [ ]:
##------- load two earthquake catalogs ----------------------##
eqcat = pd.read_csv('located_catalog_scak400km',sep =' ')
equsgs = pd.read_csv('usgs_20180501_20190831.csv')

datetime_eq = [datetime.datetime.strptime(itime,'%Y-%m-%dT%H:%M:%S.%f') for itime in eqcat['Time']]
datetime_usgs = [datetime.datetime.strptime(itime,'%Y-%m-%dT%H:%M:%S.%fZ') for itime in equsgs['time']]

In [ ]:
Direc = '2018-**-**'
datetime0 = datetime.datetime.strptime(Direc,'%Y-%m-%d')
## set up a day
l1 = 2; l2 = 8; h1 = 10; h2 = 16;
for iday in range(***):
    datetime1 = datetime0 + datetime.timedelta(days = iday)
    Direc = datetime.datetime.strftime(datetime1,'%Y-%m-%d')
    print(Direc)
    
## obtain data    
    cha = 'HH1'
    sta_name = glob.glob(Direc + '/*_HH2_180s_hf_'+str(h1)+'-'+str(h2)+'.npy')
    sta_name2 = [x.split("/")[1].split("_")[0] for x in sta_name]
    # from collections import defaultdict
    data_HH1_lf = dict.fromkeys(sta_name2)
    data_HH1_hf = dict.fromkeys(sta_name2)
 
    for ista in range(len(sta_name2)):
        with open(Direc + '/' + sta_name2[ista] +'_'+ cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
            data_HH1_lf[sta_name2[ista]]=np.load(f)
        with open(Direc + '/' + sta_name2[ista] +'_'+ cha + '_180s_hf_'+str(h1)+'-'+str(h2)+'.npy','rb') as f:
            data_HH1_hf[sta_name2[ista]]=np.load(f)            
    with open(Direc + '/EP22_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH1_rf1 = np.load(f)  
    with open(Direc + '/WD53_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH1_rf2 = np.load(f)  
    with open(Direc + '/LT20_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH1_rf3 = np.load(f)  
    with open(Direc + '/WD67_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH1_rf4 = np.load(f) 
        
    cha = 'HH2'
    data_HH2_lf = dict.fromkeys(sta_name2)
    data_HH2_hf = dict.fromkeys(sta_name2)
    for ista in range(len(sta_name2)):
        with open(Direc + '/' + sta_name2[ista] +'_'+ cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
            data_HH2_lf[sta_name2[ista]]=np.load(f)
        with open(Direc + '/' + sta_name2[ista] +'_'+ cha + '_180s_hf_'+str(h1)+'-'+str(h2)+'.npy','rb') as f:
            data_HH2_hf[sta_name2[ista]]=np.load(f)            
    with open(Direc + '/EP22_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH2_rf1 = np.load(f) 
    with open(Direc + '/WD53_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH2_rf2 = np.load(f) 
    with open(Direc + '/LT20_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH2_rf3 = np.load(f) 
    with open(Direc + '/WD67_' + cha + '_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','rb') as f:
        data_HH2_rf4 = np.load(f)         
        
## Calculate Cross-correlation, 959: the length of 1 day seismic data
    shift_day_H1_lf = dict.fromkeys(range(959))
    shift_day_H2_lf = dict.fromkeys(range(959))
    cc_day_lf = dict.fromkeys(range(959))
    cc_day_hf = dict.fromkeys(range(959))
    cc_day_rf1 = dict.fromkeys(range(959))
    cc_day_rf2 = dict.fromkeys(range(959))
    cc_day_rf3 = dict.fromkeys(range(959))
    cc_day_rf4 = dict.fromkeys(range(959))
    
    for istep in range(959):
        cc_matrix_lf = np.zeros((len(sta_name2),len(sta_name2)))
        cc_matrix_hf = np.zeros((len(sta_name2),len(sta_name2)))
        cc_matrix_rf1 = np.zeros((len(sta_name2),1))
        cc_matrix_rf2 = np.zeros((len(sta_name2),1))
        cc_matrix_rf3 = np.zeros((len(sta_name2),1))
        cc_matrix_rf4 = np.zeros((len(sta_name2),1))
        
        shift_matrix_H1_lf = np.zeros((len(sta_name2),len(sta_name2)))
        shift_matrix_H2_lf = np.zeros((len(sta_name2),len(sta_name2)))

        for ista1 in range(len(sta_name2)):     
            dataH1_lf = data_HH1_lf[sta_name2[ista1]][:,istep]
            seg1,cum_len1 = seg_cum_function(dataH1_lf)
            dataH1_hf = data_HH1_hf[sta_name2[ista1]][:,istep]
            seg_hf1,cum_len_hf1 = seg_cum_function(dataH1_hf)
            
            dataH2_lf = data_HH2_lf[sta_name2[ista1]][:,istep]
            seg2,cum_len2 = seg_cum_function(dataH2_lf)
            dataH2_hf = data_HH2_hf[sta_name2[ista1]][:,istep] 
            seg_hf2,cum_len_hf2 = seg_cum_function(dataH2_hf)
            
            if ((np.std(dataH1_lf)!=0.0 or np.std(dataH2_lf)!=0.0) and 
                (seg1>10 or seg2>10 or (len(cum_len1[0])/180)>0.3 or (len(cum_len2[0])/180)>0.3)):    
                
    ## reference station EP22
                seg_rf1,cum_len_rf1 = seg_cum_function(data_HH1_rf1[:,istep])
                seg_rf2,cum_len_rf2 = seg_cum_function(data_HH2_rf1[:,istep])
                if (seg_rf1>10 or seg_rf2>10 or (len(cum_len_rf1[0])/180)>0.3 or (len(cum_len_rf2[0])/180)>0.3): 
                    cc_rf1 = obspy.signal.cross_correlation.correlate(dataH1_lf,data_HH1_rf1[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf1,value_rf1 = obspy.signal.cross_correlation.xcorr_max(cc_rf1,abs_max=False) 

                    cc_rf2 = obspy.signal.cross_correlation.correlate(dataH2_lf,data_HH2_rf1[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf2,value_rf2 = obspy.signal.cross_correlation.xcorr_max(cc_rf2,abs_max=False)
                    cc_matrix_rf1[ista1] = zeromean([value_rf1,value_rf2])

    ## reference station WD53 
                seg_rf3,cum_len_rf3 = seg_cum_function(data_HH1_rf2[:,istep])
                seg_rf4,cum_len_rf4 = seg_cum_function(data_HH2_rf2[:,istep])
                if (seg_rf3>10 or seg_rf4>10 or (len(cum_len_rf3[0])/180)>0.3 or (len(cum_len_rf4[0])/180)>0.3):     
                    cc_rf3 = obspy.signal.cross_correlation.correlate(dataH1_lf,data_HH1_rf2[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf3,value_rf3 = obspy.signal.cross_correlation.xcorr_max(cc_rf3,abs_max=False) 

                    cc_rf4 = obspy.signal.cross_correlation.correlate(dataH2_lf,data_HH2_rf2[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf4,value_rf4 = obspy.signal.cross_correlation.xcorr_max(cc_rf4,abs_max=False)
                    cc_matrix_rf2[ista1] = zeromean([value_rf3,value_rf4])

    ## reference station LT20
                seg_rf5,cum_len_rf5 = seg_cum_function(data_HH1_rf3[:,istep])
                seg_rf6,cum_len_rf6 = seg_cum_function(data_HH2_rf3[:,istep])
                if (seg_rf5>10 or seg_rf6>10 or (len(cum_len_rf5[0])/180)>0.3 or (len(cum_len_rf6[0])/180)>0.3):      
                    cc_rf5 = obspy.signal.cross_correlation.correlate(dataH1_lf,data_HH1_rf3[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf5,value_rf5 = obspy.signal.cross_correlation.xcorr_max(cc_rf5,abs_max=False) 

                    cc_rf6 = obspy.signal.cross_correlation.correlate(dataH2_lf,data_HH2_rf3[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf6,value_rf6 = obspy.signal.cross_correlation.xcorr_max(cc_rf6,abs_max=False)
                    cc_matrix_rf3[ista1] = zeromean([value_rf5,value_rf6])

    ## reference station WD67     
                seg_rf7,cum_len_rf7 = seg_cum_function(data_HH1_rf4[:,istep])
                seg_rf8,cum_len_rf8 = seg_cum_function(data_HH2_rf4[:,istep])
                if (seg_rf7>10 or seg_rf8>10 or (len(cum_len_rf7[0])/180)>0.3 or (len(cum_len_rf8[0])/180)>0.3):  
                    cc_rf7 = obspy.signal.cross_correlation.correlate(dataH1_lf,data_HH1_rf4[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf7,value_rf7 = obspy.signal.cross_correlation.xcorr_max(cc_rf7,abs_max=False) 

                    cc_rf8 = obspy.signal.cross_correlation.correlate(dataH2_lf,data_HH2_rf4[:,istep],90,
                                                                   demean=True,normalize='naive')                        
                    shift_rf8,value_rf8 = obspy.signal.cross_correlation.xcorr_max(cc_rf8,abs_max=False)
                    cc_matrix_rf4[ista1] = zeromean([value_rf7,value_rf8])         
            
                
                for ista2 in range(ista1+1,len(sta_name2)):
                    dataH3_lf = data_HH1_lf[sta_name2[ista2]][:,istep]
                    seg3,cum_len3 = seg_cum_function(dataH3_lf)
                    dataH3_hf = data_HH1_hf[sta_name2[ista2]][:,istep]
                    seg_hf3,cum_len_hf3 = seg_cum_function(dataH3_hf)
                    
                    dataH4_lf = data_HH2_lf[sta_name2[ista2]][:,istep]
                    seg4,cum_len4 = seg_cum_function(dataH4_lf) 
                    dataH4_hf = data_HH2_hf[sta_name2[ista2]][:,istep]
                    seg_hf4,cum_len_hf4 = seg_cum_function(dataH4_hf)
                    
                    if ((np.std(dataH3_lf)!=0.0 or np.std(dataH4_lf)!=0.0) and 
                        (seg3>10 or seg4>10 or (len(cum_len3[0])/180)>0.3 or (len(cum_len4[0])/180)>0.3)):
                        
    ## cc values & shifts: low pass filterred data
                        cc1 = obspy.signal.cross_correlation.correlate(dataH1_lf,dataH3_lf,20,
                                                                       demean=True,normalize='naive')                        
                        shift1,value1 = obspy.signal.cross_correlation.xcorr_max(cc1,abs_max=False)                        
                        shift_matrix_H1_lf[ista1,ista2] = shift1
                        cc2 = obspy.signal.cross_correlation.correlate(dataH2_lf,dataH4_lf,20,
                                                                       demean=True,normalize='naive')                        
                        shift2,value2 = obspy.signal.cross_correlation.xcorr_max(cc2,abs_max=False)                        
                        shift_matrix_H2_lf[ista1,ista2] = shift2  
                        cc_matrix_lf[ista1,ista2] = max([value1,value2])
                        
                    if ((np.std(dataH3_lf)!=0.0 or np.std(dataH4_lf)!=0.0) and 
                        (seg_hf3>10 or seg_hf4>10 or (len(cum_len_hf3[0])/180)>0.3 or (len(cum_len_hf4[0])/180)>0.3) and
                        (seg_hf1>10 or seg_hf2>10 or (len(cum_len_hf1[0])/180)>0.3 or (len(cum_len_hf2[0])/180)>0.3)):                        
    ## cc values & shifts: high pass filterred data      
    
                        cc3 = obspy.signal.cross_correlation.correlate(dataH1_hf,dataH3_hf,20,
                                                                       demean=True,normalize='naive')                        
                        shift3,value3 = obspy.signal.cross_correlation.xcorr_max(cc3,abs_max=False)                        
                        cc4 = obspy.signal.cross_correlation.correlate(dataH2_hf,dataH4_hf,20,
                                                                       demean=True,normalize='naive')                        
                        shift4,value4 = obspy.signal.cross_correlation.xcorr_max(cc4,abs_max=False)                      
                        cc_matrix_hf[ista1,ista2] = zeromean([value3,value4])     
                    
        shift_day_H1_lf[istep] = shift_matrix_H1_lf; 
        shift_day_H2_lf[istep] = shift_matrix_H2_lf;
        cc_day_lf[istep] = cc_matrix_lf;   
        cc_day_hf[istep] = cc_matrix_hf; 
        cc_day_rf1[istep] = cc_matrix_rf1; cc_day_rf2[istep] = cc_matrix_rf2; 
        cc_day_rf3[istep] = cc_matrix_rf3; cc_day_rf4[istep] = cc_matrix_rf4; 

# save data
    with open(Direc + '/cc_day_lf_'+str(l1)+'-'+str(l2)+'_'+str(h1)+'-'+str(h2)+'.pkl','wb') as f1:
        pickle.dump(cc_day_lf,f1)
    with open(Direc + '/cc_day_hf_'+str(l1)+'-'+str(l2)+'_'+str(h1)+'-'+str(h2)+'.pkl',"wb") as f1:
        pickle.dump(cc_day_hf,f1)
    with open(Direc + '/cc_day_rf_EP22_'+str(l1)+'-'+str(l2)+'.pkl',"wb") as f1:
        pickle.dump(cc_day_rf1,f1)
    with open(Direc + '/cc_day_rf_WD53_'+str(l1)+'-'+str(l2)+'.pkl',"wb") as f1:
        pickle.dump(cc_day_rf2,f1)
    with open(Direc + '/cc_day_rf_LT20_'+str(l1)+'-'+str(l2)+'.pkl',"wb") as f1:
        pickle.dump(cc_day_rf3,f1)
    with open(Direc + '/cc_day_rf_WD67_'+str(l1)+'-'+str(l2)+'.pkl',"wb") as f1:
        pickle.dump(cc_day_rf4,f1)
        
    with open(Direc + "/shift_day_H1_lf_2-8_10-16.pkl","wb") as f1:
        pickle.dump(shift_day_H1_lf,f1)
    with open(Direc + "/shift_day_H2_lf_2-8_10-16.pkl","wb") as f1:
        pickle.dump(shift_day_H2_lf,f1)    
    with open(Direc + '/station_name.npy','wb') as f:
        np.save(f,sta_name2)  
